In [4]:
# %% imports
# libraries
import torch
from torch.utils.data import Dataset,DataLoader
import torch.optim as optim
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt
import glob
import os
import numpy as np

# local imports
import MNIST_dataloader
# import autoencoder_template
from config_file import data_loc
from matplotlib import pyplot as plt
from MNIST_dataloader import create_dataloaders

# Week 1

### Exercise 1 - ISTA

##### (a)[3 pt]Create a Python function that implements ISTA for MNIST denoising. 
Make sure the function takes as inputs: step size μ, shrinkage parameter λ, number of
iteration K, and measurements (batch of MNIST images) y. It should output the
final batch of reconstructions xK.

hint: the images are normalized between -1 and 1, which means the background of the
images is not at zero. Think about how to still aply ISTA correctly.

##### (b) [2 pt] Use your ISTA algorithm on 10 example images. 
Show them in a figure consisting of 10 collumns (1 per digit) and 3 rows. Row 1 should contain the noisy
measurement, row 2 the reconstruction, and row 3 the actual ground truth target.
You will have to play with the values for μ, λ, and K. Explain what you see.

##### (c) [1 pt] To get a numerical estimate for the performance of ISTA, run it on the entire test set and report the resulting mean squared error (mse).